In [1]:
import os
import sys
import json
import torch

In [2]:
# inspect /bodmas_batches/train/batch_0139.pt 

file = '/data/saranyav/gcn_new/bodmas_batches_test/train/batch_0139.pt'


In [4]:
import torch

# Path to the .pt file
file_path = "/data/saranyav/gcn_new/bodmas_batches_test/train/batch_0139.pt"

# Load the .pt file
data = torch.load(file_path)

# Inspect the contents
print("Keys in the loaded file (if dictionary):")
if isinstance(data, dict):
    for key, value in data.items():
        print(f"Key: {key}, Type: {type(value)}, Shape/Length: {getattr(value, 'shape', len(value) if hasattr(value, '__len__') else 'N/A')}")
elif isinstance(data, list):
    print(f"The file contains a list with {len(data)} items.")
else:
    print(f"Type of data: {type(data)}")

# Optional: Preview specific content
print("\nExample content:")
if isinstance(data, dict):
    first_key = next(iter(data))
    print(f"First key: {first_key}, Content: {data[first_key]}")
elif isinstance(data, list):
    print(data[0])

Keys in the loaded file (if dictionary):
The file contains a list with 59 items.

Example content:
Data(x=[7366, 14], edge_index=[2, 7795], edge_attr=[3116, 1], num_nodes=7366, sha='7f2d1e70b3dcb0c8e85a8e7c88ef2594c2f371055ba49e0941764c9df9cc798e', timestamp='2019-08-29 09:59:25 UTC', family='autoit', malware_type='trojan')


/tmp/ipykernel_3856677/2074203216.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


In [10]:
import torch



# Function to check for family='none'
def check_family_none(data):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict) or isinstance(value, list):
                # Recursively inspect nested dictionaries or lists
                result = check_family_none(value)
                if result:
                    return result
            elif hasattr(value, 'family') and getattr(value, 'family', None) == 'none':
                return True
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, dict) or isinstance(item, list):
                # Recursively inspect nested lists or dictionaries
                result = check_family_none(item)
                if result:
                    return result
            elif hasattr(item, 'family') and getattr(item, 'family', None) == 'none':
                return True
    elif hasattr(data, 'family') and getattr(data, 'family', None) == 'none':
        return True
    return False

# Check for family='none'
found = check_family_none(data)
if found:
    print("A key or attribute with family='none' exists in the data.")
else:
    print("No key or attribute with family='none' was found in the data.")

No key or attribute with family='none' was found in the data.


In [ ]:
import os
import torch

# Directory containing the .pt files
directory =  "/data/saranyav/gcn_new/bodmas_batches/train"


# Function to check if family='none' exists in a data object
def has_family_none(data):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                if has_family_none(value):
                    return True
            elif hasattr(value, 'family') and getattr(value, 'family', None) == 'none':
                return True
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                if has_family_none(item):
                    return True
            elif hasattr(item, 'family') and getattr(item, 'family', None) == 'none':
                return True
    elif hasattr(data, 'family') and getattr(data, 'family', None) == 'none':
        return True
    return False

# Iterate through all .pt files in the directory
for root, _, files in os.walk(directory):
    for file in files:
        if file.endswith(".pt"):
            file_path = os.path.join(root, file)
            try:
                # Load the .pt file
                data = torch.load(file_path)
                # Check for family='none'
                if has_family_none(data):
                    print(f"Found family='none' in file: {file_path}")
            except Exception as e:
                print(f"Could not process file {file_path}: {e}")